In [6]:
import requests
import json
import re
from datetime import datetime
from azure.cosmos import CosmosClient
import uuid

nombre_software = 'MySQL'  # Coloca el nombre de tu software
MITRE_API_URL = 'https://cve.circl.lu/api/cve/'  # Uso API Mitre
cve_ids = ['CVE-2016-3333']  # CVEs

# Configuración de la conexión a Cosmos DB
url = "https://andresdia02.documents.azure.com:443/"
clave_maestra = "ovpWKq3QnVf76g2KKnwQb2FArJz9X6lroZ3bqhvqXIkdE3rgxObPWRVWxNU6yWEwDxER5Imd983HACDbCIbnUQ=="
base_datos_id = "base1"
contenedor_id = "nombre_del_contenedor"

# Crear un cliente de Cosmos DB
client = CosmosClient(url, credential=clave_maestra)

# Verificar si la base de datos existe
try:
    database = client.get_database_client(base_datos_id)
    print("La base de datos ya existe.")
except Exception as e:
    # Crear la base de datos si no existe
    database = client.create_database_if_not_exists(id=base_datos_id)
    print("Se ha creado la base de datos.")

# Verificar si el contenedor existe
try:
    container = database.get_container_client(contenedor_id)
    print("El contenedor ya existe.")
except Exception as e:
    # Crear el contenedor si no existe
    container = database.create_container_if_not_exists(id=contenedor_id, partition_key=("/partitionKey",))
    print("Se ha creado el contenedor.")

# Funcion de consumo a la api de MITRE
def obtener_datos_mitre(cve_id):
    """
    Busca información sobre la vulnerabilidad en MITRE y extrae hasta tres CVEs relacionados si los hay.
    """
    url = f'{MITRE_API_URL}{cve_id}'
    try:
        print(f"Buscando información de la vulnerabilidad en MITRE: {cve_id}")
        response = requests.get(url)
        response.raise_for_status()
        cve_data = response.json()

        description = cve_data.get('summary', 'Descripción no disponible')

        # Buscar otros CVEs mencionados en la descripción
        cves_encontrados = re.findall(r'CVE-\d{4}-\d{4,7}', description)

        # Limitar a los primeros tres CVEs encontrados
        cves_relacionados = cves_encontrados[:3]

        return {
            "id": str(uuid.uuid4()),  # Generar un ID único para el documento
            "CVE": cve_id,
            "Descripcion": description,
            "CVEs_Relacionados": cves_relacionados
        }
    except requests.exceptions.RequestException as e:
        print(f'Error al obtener la información de la vulnerabilidad en MITRE: {e}')
        return None

datos_json = {
    "nombre_software": nombre_software,
    "date": datetime.now().strftime("%Y-%m-%d"),
    "vulnerabilidades": []
}

# Obtener datos de MITRE y guardar en JSON
for cve_id in cve_ids:
    cve_info = obtener_datos_mitre(cve_id)
    if cve_info:
        datos_json["vulnerabilidades"].append(cve_info)

# Insertar datos en el contenedor de Cosmos DB
for dato in datos_json["vulnerabilidades"]:
    container.create_item(body=dato)

print(f"Proceso terminado. Los resultados de '{nombre_software}' se han guardado en el contenedor de Azure Cosmos DB.")


La base de datos ya existe.
El contenedor ya existe.
Buscando información de la vulnerabilidad en MITRE: CVE-2016-3333
Proceso terminado. Los resultados de 'MySQL' se han guardado en el contenedor de Azure Cosmos DB.
